In [1]:
#
# Example - Using uvatradier to fetch quote data for a given symbol
#

import os, dotenv; import pandas as pd;
from uvatradier import Tradier, Quotes

# Create .env file within current working directory.
# Add the following to your .env file. Specify the correct account number and authorization token for your quote.
# 	tradier_acct = <ACCOUNT_NUMBER>
#	tradier_token = <AUTH_TOKEN>

dotenv.load_dotenv()

ACCOUNT_NUMBER = os.getenv('tradier_acct')
AUTH_TOKEN = os.getenv('tradier_token')

# Initializing new Quotes object
quotes = Quotes(ACCOUNT_NUMBER, AUTH_TOKEN);

DOW_30_TICKER = [
    "AAPL",
    "MSFT",
    "JPM",
    "V",
    "RTX",
    "PG",
    "GS",
    "NKE",
    "DIS",
    "AXP",
    "HD",
    "INTC",
    "WMT",
    "IBM",
    "MRK",
    "UNH",
    #"KO",
    "CAT",
    "TRV",
    "JNJ",
    "CVX",
    "MCD",
    "VZ",
    "CSCO",
    "XOM",
    "BA",
    "MMM",
    "PFE",
    "WBA",
    "DD",
]

# data = pd.DataFrame(quotes.get_historical_quotes('KO', start_date='2022-10-01', end_date='2023-10-01'))

# for symb in DOW_30_TICKER:
#     data_to_append = pd.DataFrame(quotes.get_historical_quotes(symb, start_date='2022-10-01', end_date='2023-10-01'))
#     data = pd.concat([data, data_to_append], ignore_index=True)

data = pd.read_csv("DOW30data.csv")

print(data);

# data['close_change'] = data['close'].pct_change()
# data['volume_change'] = data['volume'].pct_change()\
close_change = data['close'].pct_change();
volume_change = data['volume'].pct_change();

hello, world!
            date   open    high     low  close    volume
0     2022-10-03  56.36  56.850  56.060  56.65  16277620
1     2022-10-04  56.95  57.810  56.460  56.78  18861431
2     2022-10-05  56.50  56.650  55.510  56.24  14324851
3     2022-10-06  56.00  56.100  54.870  55.03  17192843
4     2022-10-07  54.99  55.090  54.250  54.51  11479799
...          ...    ...     ...     ...    ...       ...
7495  2023-09-25  72.40  73.630  72.400  73.36   1790625
7496  2023-09-26  72.70  73.550  72.360  72.78   2517575
7497  2023-09-27  73.14  74.010  72.830  73.47   2018135
7498  2023-09-28  73.46  74.855  73.460  74.54   2122866
7499  2023-09-29  75.11  75.460  74.465  74.59   2315988

[7500 rows x 6 columns]


In [2]:
# import csv

# # Specify the file name
# file_name = 'DOW30data.csv'

# # Write the DataFrame to a CSV file
# data.to_csv(file_name, index=False)

# print(f'Data has been written to {file_name}')

In [3]:
print(close_change)

0            NaN
1       0.002295
2      -0.009510
3      -0.021515
4      -0.009449
          ...   
7495    0.007139
7496   -0.007906
7497    0.009481
7498    0.014564
7499    0.000671
Name: close, Length: 7500, dtype: float64


In [4]:
volume_change

0            NaN
1       0.158734
2      -0.240522
3       0.200211
4      -0.332292
          ...   
7495   -0.074943
7496    0.405976
7497   -0.198381
7498    0.051895
7499    0.090972
Name: volume, Length: 7500, dtype: float64

In [10]:
# Given the close_change Series (as per your example)
# close_change = pd.Series([None, 0.002295, -0.009510, -0.021515, -0.009449], name='close')  # truncated for brevity

# Replace NaN values with the mean of the non-NaN values
close_change_filled = close_change.fillna(close_change.mean())
volume_change_filled = volume_change.fillna(volume_change.mean())

# print(close_change_filled)
# print(volume_change_filled)


logit_data = pd.DataFrame({'close':close_change_filled, 'volume':volume_change_filled});
print(logit_data)


         close     volume
0     0.001657  36.436996
1     0.002295   0.158734
2    -0.009510  -0.240522
3    -0.021515   0.200211
4    -0.009449  -0.332292
...        ...        ...
7495  0.007139  -0.074943
7496 -0.007906   0.405976
7497  0.009481  -0.198381
7498  0.014564   0.051895
7499  0.000671   0.090972

[7500 rows x 2 columns]


In [14]:
logit_data.query("(volume > 10) or (volume < -10)")

,close,volume
0,0.001657,36.436996
486,-0.018602,9333.799298
736,-0.014349,53227.733333
2000,-0.736069,12.611002
3000,-0.910743,11.589635
3236,0.004134,69157.205000
5986,-0.007218,139159.880000
7236,0.013060,1806.040041


In [6]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# # Assuming these are your Series (based on provided data)
# close = pd.Series([
#     -0.000009, 0.002295, -0.009510, -0.021515, -0.009449, # truncated for brevity
#     -0.010417, -0.008246, -0.010260, -0.002502, 0.003046
# ])

# volume = pd.Series([
#     0.057599, 0.158734, -0.240522, 0.200211, -0.332292, # truncated for brevity
#     -0.156129, -0.094194, 0.229716, -0.093092, -0.017077
# ])

# Convert close to binary. Assuming a positive change is 1 and non-positive is 0.
logit_data['close_binary'] = (logit_data['close'] > 0).astype(int)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(logit_data[['volume']], logit_data['close_binary'], test_size=0.2, random_state=42)

# Create a logistic regression model
model = LogisticRegression()

# Fit the model
model.fit(X_train, y_train)

# Get the score of the model
score = model.score(X_test, y_test)

print(f"Model Accuracy: {score*100:.2f}%")


Model Accuracy: 52.40%
